In [1]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

/opt/conda/envs/jph/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## Реализуем метрику из задачи

In [2]:
def metric(true, pred, coef=1000):
    assert len(true) == len(pred)
    error = 0
    for i in range(len(true)):
        if true[i] > pred[i]:
            error += true[i] - pred[i]
        else:
            error += (pred[i] - true[i]) * coef
    return -round(error / len(true) / 1_000_000_000)

## Прочитаем и подготовим данные

In [3]:
df_train = pd.read_csv('/home/jovyan/work/train.csv')

In [4]:
df_train.head()

,id,day 1,day 2,day 3,day 4,day 5,day 6,day 7,day 8,day 9,...,day 81,day 82,day 83,day 84,day 85,day 86,day 87,day 88,day 89,day 90
0,6602185159948,569369144437,152818843344,-15490421929,-406624212,800362695678,2306966308406,1917859437929,1075925676467,277630770065,...,-23536698564,1524999469315,1668473296482,-55911238122,1105906678699,1134120612066,-26829699967,-19492512239,1850461161279,-1781621104800
1,1663310387493,-104722473995,-29275417695,423353969042,416864670779,-323866168258,-59323097843,58355171506,250838843414,-184607036341,...,529867756317,-245569015020,-13050008306,1122739774405,-358972410001,-294465761456,289760186052,588830896355,-21661542192,-153747680862
2,3604676372924,-42697753432,154940888256,5290701628,2915610080,-241412656598,-2172755647,-60115526657,99210264232,281086593365,...,32773187167,-347787851371,172520154368,46099833867,-297776587790,200380940252,153279754373,42773997259,5607112117,4789271560371
3,3517792576177,735945593435,877636042341,-5241075542,-1817826687,1008399822616,996548931508,471592893006,479393037398,246618026800,...,4227540489,855279732927,808871471815,-8997018250,627399608353,223093070252,-25844611162,-9955925825,603978495789,366344015281
4,6892195368446,697850269572,980815916948,-11470196952,-21103754202,1207799272628,1202816134496,559646086486,1058635690664,122661998719,...,-17229675906,940502608045,728437869463,13477840270,1455297859610,409612399843,-6730004331,19193466540,1163355637082,-138405088616


In [5]:
df_train.shape

(20000, 91)

In [6]:
X = df_train[[f'day {x}' for x in range(1, 90)]]
Y = df_train['day 90']

In [7]:
X.head()

,day 1,day 2,day 3,day 4,day 5,day 6,day 7,day 8,day 9,day 10,...,day 80,day 81,day 82,day 83,day 84,day 85,day 86,day 87,day 88,day 89
0,569369144437,152818843344,-15490421929,-406624212,800362695678,2306966308406,1917859437929,1075925676467,277630770065,-1039738738,...,-19148904713,-23536698564,1524999469315,1668473296482,-55911238122,1105906678699,1134120612066,-26829699967,-19492512239,1850461161279
1,-104722473995,-29275417695,423353969042,416864670779,-323866168258,-59323097843,58355171506,250838843414,-184607036341,371352008249,...,330976588040,529867756317,-245569015020,-13050008306,1122739774405,-358972410001,-294465761456,289760186052,588830896355,-21661542192
2,-42697753432,154940888256,5290701628,2915610080,-241412656598,-2172755647,-60115526657,99210264232,281086593365,64728036374,...,169669285257,32773187167,-347787851371,172520154368,46099833867,-297776587790,200380940252,153279754373,42773997259,5607112117
3,735945593435,877636042341,-5241075542,-1817826687,1008399822616,996548931508,471592893006,479393037398,246618026800,-5322931641,...,-25780592418,4227540489,855279732927,808871471815,-8997018250,627399608353,223093070252,-25844611162,-9955925825,603978495789
4,697850269572,980815916948,-11470196952,-21103754202,1207799272628,1202816134496,559646086486,1058635690664,122661998719,3419722951,...,23074058185,-17229675906,940502608045,728437869463,13477840270,1455297859610,409612399843,-6730004331,19193466540,1163355637082


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

## Обучим модель

In [9]:
%%time
model = XGBRegressor(n_estimators=100)
model.fit(X_train, Y_train)
pred = model.predict(X_test)

/opt/conda/envs/jph/lib/python3.10/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


CPU times: user 3min 52s, sys: 2.65 s, total: 3min 54s
Wall time: 3min 59s


In [10]:
metric(Y_test.tolist(), pred)

-105744

In [11]:
# Попробуем уменьшить предсказания, чтобы оптимизировать ассиметричную метрику
metric(Y_test.tolist(), [0.95 * x for x in pred])

-98051

## Сделаем предсказание

In [ ]:
df_test = pd.read_csv('/home/jovyan/work/test.csv')

In [ ]:
X_pred = df_test[[f'day {x}' for x in range(2, 91)]]

In [ ]:
X_pred.shape

In [ ]:
pred = model.predict(X_pred)

In [ ]:
df_test['day 91'] = pred * 0.95

In [ ]:
df_test.head()

In [ ]:
df_test[['id', 'day 91']].to_csv('/home/jovyan/work/samplesubmission.csv', index=False)